## Irvine Current Planning Projects

In [13]:
import sys
sys.path.append('/Projects/regionintelligenceai/map/')

In [15]:
#from src.const import IRVINE_MAJOR_PLANNING_URL

#url = IRVINE_MAJOR_PLANNING_URL
url = "https://www.cityofirvine.org/community-development/pending-major-development-applications" 
print(url)

https://www.cityofirvine.org/community-development/pending-major-development-applications


In [31]:
#from src.driver_config import get_chrome_driver
import time
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re
from datetime import datetime
from pathlib import Path
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
#from src.logger import get_console_logger

#logger = get_console_logger()

def get_chrome_driver():
    """Initializes and returns a Selenium Chrome web driver using ChromeDriverManager."""
    options = Options()
    return webdriver.Chrome()

#driver = get_chrome_driver()

In [25]:
#from src.paths import RAW_DATA_DIR, PROCESSED_DATA_DIR, COMPANY_DATA_DIR
from urllib.parse import urlparse
import requests
import pdfplumber

class IrvineScraper:
    current_date = datetime.now().strftime("%Y-%m-%d")

    def __init__(self, driver):
        self.driver = driver
    
    def connect(self, url):
        self.driver.get(url)
        print(self.driver.title)

    def scrape_pdf(self):
        try:
            # Click the link to open in a new tab
            link = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Discretionary Project List"))
            )
            link.click()
            print("Navigated to the Discretionary Project List")

            # Switch to the new tab and wait for it to load
            WebDriverWait(self.driver, 10).until(lambda d: len(d.window_handles) > 1)
            self.driver.switch_to.window(self.driver.window_handles[1])
            time.sleep(5)  # Adjust time as needed

            # Get the URL of the new tab
            pdf_url = self.driver.current_url

            # Sanitize the URL if it has been duplicated
            if pdf_url.count('BlobID=') > 1:
                pdf_url = pdf_url.split('&')[0]

            print(f"PDF should be available at: {pdf_url}")

            # Send a GET request to the PDF URL using requests
            response = requests.get(pdf_url)
            if response.status_code == 200:
                # Ensure the 'raw_data' directory exists
                raw_data_dir = 'raw_data'
                if not os.path.exists(raw_data_dir):
                    os.makedirs(raw_data_dir)
                
                # Extract the filename from the URL and create a valid path
                filename = os.path.basename('output.pdf')
                file_path = os.path.join(raw_data_dir, filename)

                # Write the PDF to a file
                with open(file_path, 'wb') as pdf_file:
                    pdf_file.write(response.content)
                print(f"PDF has been saved to {file_path}")
            else:
                print(f"Failed to download the PDF. Status code: {response.status_code}")

            # Close the PDF tab
            self.driver.close()

            # Switch back to the original tab
            self.driver.switch_to.window(self.driver.window_handles[0])
            
        except Exception as e:
            print(f"An error occurred: {e}")

            """
            response = requests.get(pdf_url)
            if response.status_code == 200:
                # Parse the filename from the URL
                a = urlparse(pdf_url)
                filename = os.path.basename(a.path)
                # Make sure the RAW_DATA_DIR exists
                if not os.path.exists(RAW_DATA_DIR):
                    os.makedirs(RAW_DATA_DIR)
                # Write the PDF content to a file in RAW_DATA_DIR
                with open(os.path.join(RAW_DATA_DIR, filename), 'wb') as f:
                    f.write(response.content)
                print(f"PDF has been saved to {os.path.join(RAW_DATA_DIR, filename)}")
            else:
                print(f"Failed to download the PDF. Status code: {response.status_code}")
            
            # Close the PDF tab
            self.driver.close()

            # Switch back to the original tab
            self.driver.switch_to.window(self.driver.window_handles[0])
            
        except Exception as e:
            print(f"An error occurred: {e}")
            """
    
    def wait_for_download_to_complete(self, file_pattern):
        """ Wait for the download to complete """
        print("Waiting for download to complete...")
        timeout = 120  # 2 minutes timeout
        while timeout > 0:
            files = os.listdir()
            if any(f.startswith(file_pattern) for f in files):
                print("Download completed.")
                return
            time.sleep(1)
            timeout -= 1
        print("Download did not complete within the allotted time.")

    @staticmethod
    def parse_pdf(pdf_filename):
        try:
            with pdfplumber.open(pdf_filename) as pdf:
                full_text = "\n".join(page.extract_text() or "" for page in pdf.pages)
            print(full_text)
        except Exception as e:
            print(f"An error occurred while parsing the PDF: {e}")

    

# Example usage
url = 'https://www.cityofirvine.org/community-development/pending-major-development-applications'
driver = get_chrome_driver()
scraper = IrvineScraper(driver)
scraper.connect(url)
scraper.scrape_pdf()
driver.quit()

Pending Major Development Applications | City of Irvine
Navigated to the Discretionary Project List
PDF should be available at: https://legacy.cityofirvine.org/civica/filebank/blobdload.asp?BlobID=11038
PDF has been saved to raw_data\output.pdf


In [27]:
import pdfplumber
import os

def parse_pdf(pdf_filename):
    raw_data_dir = 'raw_data'
    file_path = os.path.join(raw_data_dir, pdf_filename)

    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"The file {pdf_filename} does not exist in the directory {raw_data_dir}.")
        return

    try:
        # Open the PDF file
        with pdfplumber.open(file_path) as pdf:
            full_text = ""
            # Iterate through each page
            for page in pdf.pages:
                # Extract text from the page
                text = page.extract_text()
                if text:
                    full_text += text + "\n"

        # Here you can do something with the full_text
        # For now, let's just print it
        print(full_text)

    except Exception as e:
        print(f"An error occurred while parsing the PDF: {e}")

# Example usage:
parse_pdf("output.pdf")


CURRENT DISCRETIONARY PROJECTS UNDER REVIEW
Submittal Date File # PA Project Description Schedule Decision Maker Planner
1 Ethel Coplen Way
8/24/2023 00912425-PCPM 19 Add wedding venue to Rancho San Joaquin Golf Course TBD ZA Hernan DeSantos
21 California
8/23/2023 00912351-PPD 24 Convert tot lot to a BBQ area TBD Staff Eric Martin
PA 51 D1 South
8/23/2023 00912338-PPA 51 Review of a Preliminary WQMP for District 1 South TBD Staff Sherman Jones
17850 Von Karman Avenue
8/21/2023 00912174-PCPM 36 Minor Modification to approved Home 2 Suites Hotel CUP TBD Staff Ann Wuu
D1 South - Retail
8/21/2023 00912171-PMPC 51 Commercial Master Plan for VTPM 2023-142 Lots 1-4 to allow a commercial retail 11/16/2023 PC Sherman Jones
center
6320 Irvine Blvd
8/17/2023 00912012-PCPM 9 Major CUP Modification to upgrade liquor license for sale of distilled spirits TBD PC Eric Martin
9560 Research Dr
8/17/2023 00911983-PCPM 34 CUP for warehouse and sales outlet in PA 34 TBD ZA Eric Martin
Pacifica & Gateway
M

In [29]:
import pdfplumber
import os
import pandas as pd
import re

# Function to check if the line contains date format
def contains_date(s):
    return bool(re.match(r'\d{1,2}/\d{1,2}/\d{4}', s))

# Function to extract the schedule, decision maker, and planner from the details
def extract_schedule_decision_planner(details):
    # Indices for 'Schedule', 'Decision Maker', and 'Planner' assuming they are at the end
    # The -1 index will fetch the last word from the list which is assumed to be 'Planner'
    planner_index = -1
    # The -2 index will fetch the second last word from the list which is assumed to be 'Decision Maker'
    decision_maker_index = -2
    # The -3 index will fetch the third last word from the list which is assumed to be 'Schedule'
    schedule_index = -3
    
    # We want everything before 'Schedule' to be part of the project description
    project_description = ' '.join(details[:schedule_index])
    schedule = details[schedule_index]
    decision_maker = details[decision_maker_index]
    planner = details[planner_index]
    
    return project_description, schedule, decision_maker, planner

# Function to process text and extract project data
def process_text(text):
    lines = text.strip().split("\n")
    projects = []
    project_title = None

    for line in lines:
        if line.startswith("CURRENT DISCRETIONARY PROJECTS UNDER REVIEW"):
            continue
        if not contains_date(line.strip()):
            project_title = line.strip()
        else:
            details = line.strip().split()
            if len(details) > 3 and project_title:
                project_description, schedule, decision_maker, planner = extract_schedule_decision_planner(details)
                project_data = {
                    'Submittal Date': details[0],
                    'File #': details[1],
                    'PA': details[2],
                    'Project Description': project_description,
                    'Schedule': schedule,
                    'Decision Maker': decision_maker,
                    'Planner': planner,
                    'Project Title': project_title
                }
                projects.append(project_data)
                project_title = None  # Reset project title for the next block

    return projects

# Function to create a DataFrame from a list of dictionaries
def create_dataframe(projects):
    return pd.DataFrame(projects)

def parse_pdf(pdf_filename):
    raw_data_dir = 'raw_data'
    file_path = os.path.join(raw_data_dir, pdf_filename)

    if not os.path.exists(file_path):
        print(f"The file {pdf_filename} does not exist in the directory {raw_data_dir}.")
        return

    try:
        with pdfplumber.open(file_path) as pdf:
            full_text = ""
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    full_text += text + "\n"
        return full_text
    except Exception as e:
        print(f"An error occurred while parsing the PDF: {e}")
        return None

# Example usage:
pdf_text = parse_pdf("output.pdf")
if pdf_text:
    project_data = process_text(pdf_text)
    project_df = create_dataframe(project_data)
    print(project_df.head())  # Print the first few rows to check the DataFrame


  Submittal Date         File #  PA  \
0      8/24/2023  00912425-PCPM  19   
1      8/23/2023   00912351-PPD  24   
2      8/23/2023   00912338-PPA  51   
3      8/21/2023  00912174-PCPM  36   
4      8/21/2023  00912171-PMPC  51   

                                 Project Description Schedule Decision Maker  \
0  8/24/2023 00912425-PCPM 19 Add wedding venue t...       ZA         Hernan   
1  8/23/2023 00912351-PPD 24 Convert tot lot to a...    Staff           Eric   
2  8/23/2023 00912338-PPA 51 Review of a Prelimin...    Staff        Sherman   
3  8/21/2023 00912174-PCPM 36 Minor Modification ...    Staff            Ann   
4  8/21/2023 00912171-PMPC 51 Commercial Master P...       PC        Sherman   

    Planner            Project Title  
0  DeSantos       1 Ethel Coplen Way  
1    Martin            21 California  
2     Jones           PA 51 D1 South  
3       Wuu  17850 Von Karman Avenue  
4     Jones        D1 South - Retail  


In [39]:
project_df.rename(columns={'Submittal Date': 'recentUpdate', 'File #': 'file_number', 'PA': 'pa', 'Project Description': 'description', 'Schedule': 'schedule', 'Decision Maker': 'plannerFirst', 'Planner': 'plannerLast', 'Project Title': 'projectName'}, inplace=True)
# Function to clean the description
def clean_description(description):
    # Remove 'TBD'
    description = description.replace('TBD', '')
    # Remove the date using regex
    description = re.sub(r'\d+/\d+/\d+', '', description).strip()
    # Remove file number using regex
    description = re.sub(r'\b\d{8}-[A-Z]{3,4}\b', '', description).strip()
    return description

# Apply the cleaning function to the 'description' column
project_df['description'] = project_df['description'].apply(clean_description)

# Drop any rows where 'description' is now empty, if required
project_df = project_df[project_df['description'] != '']

# Print the cleaned DataFrame to verify
print(project_df.head())

project_df['city'] = 'Irvine'
project_df['planner'] = project_df['plannerFirst'] + ' ' + project_df['plannerLast']

def fix_project_title(row):
    # Check if the project_title is the exact problematic string
    if row['project_title'] == 'Submittal Date File # PA Project Description Schedule Decision Maker Planner':
        # Replace with 'PA #' followed by the pa value
        return 'PA ' + str(row['pa'])
    else:
        # If not, return the original project_title
        return row['project_title']

# Apply the function to each row in the DataFrame
project_df['project_title'] = project_df.apply(fix_project_title, axis=1)

# Check the DataFrame to confirm the changes
print(project_df[['pa', 'project_title']])

  recentUpdate    file_number  pa  \
0    8/24/2023  00912425-PCPM  19   
1    8/23/2023   00912351-PPD  24   
2    8/23/2023   00912338-PPA  51   
3    8/21/2023  00912174-PCPM  36   
4    8/21/2023  00912171-PMPC  51   

                                                                         description  \
0                             19 Add wedding venue to Rancho San Joaquin Golf Course   
1                                                   24 Convert tot lot to a BBQ area   
2                               51 Review of a Preliminary WQMP for District 1 South   
3                          36 Minor Modification to approved Home 2 Suites Hotel CUP   
4  51 Commercial Master Plan for VTPM 2023-142 Lots 1-4 to allow a commercial retail   

  schedule plannerFirst plannerLast            project_title    city  
0       ZA       Hernan    DeSantos       1 Ethel Coplen Way  Irvine  
1    Staff         Eric      Martin            21 California  Irvine  
2    Staff      Sherman       Jones

In [43]:
## If the data is submitted within the last thirty days, the project is considered active. Otherwise, it is considered unknown
# Add a status column based on submittal date
from datetime import datetime, timedelta

def determine_status(submittal_date_str):
    submittal_date = datetime.strptime(submittal_date_str, '%m/%d/%Y')
    if datetime.now() - submittal_date <= timedelta(days=30):
        return 'In-Review'
    else:
        return 'Unknown'  # or any other default status

# Assuming 'project_df' is your DataFrame and it has a column 'Submittal Date' with dates in the format 'mm/dd/yyyy'
project_df['status'] = project_df['recentUpdate'].apply(determine_status)

project_df

,recentUpdate,file_number,pa,description,schedule,plannerFirst,plannerLast,project_title,city,Status
0,8/24/2023,00912425-PCPM,19,19 Add wedding venue to Rancho San Joaquin Golf Course,ZA,Hernan,DeSantos,1 Ethel Coplen Way,Irvine,Unknown
1,8/23/2023,00912351-PPD,24,24 Convert tot lot to a BBQ area,Staff,Eric,Martin,21 California,Irvine,Unknown
2,8/23/2023,00912338-PPA,51,51 Review of a Preliminary WQMP for District 1 South,Staff,Sherman,Jones,PA 51 D1 South,Irvine,Unknown
3,8/21/2023,00912174-PCPM,36,36 Minor Modification to approved Home 2 Suites Hotel CUP,Staff,Ann,Wuu,17850 Von Karman Avenue,Irvine,Unknown
4,8/21/2023,00912171-PMPC,51,51 Commercial Master Plan for VTPM 2023-142 Lots 1-4 to allow a commercial retail,PC,Sherman,Jones,D1 South - Retail,Irvine,Unknown
5,8/17/2023,00912012-PCPM,9,9 Major CUP Modification to upgrade liquor license for sale of distilled spirits,PC,Eric,Martin,6320 Irvine Blvd,Irvine,Unknown
6,8/17/2023,00911983-PCPM,34,34 CUP for warehouse and sales outlet in PA 34,ZA,Eric,Martin,9560 Research Dr,Irvine,Unknown
7,8/17/2023,00911981-PDA,33,"33 DA for Pacifica Place in PA 33 PC,",CC,Victor,Mendez,PA #33,Irvine,Unknown
8,8/16/2023,00911921-PCPM,36,36 CUP for a small medical office space within an existing laboratory,ZA,Victor,Mendez,17672 Cowan,Irvine,Unknown
9,8/11/2023,00911531-PSS,36,36 Sign Program with AR for Reece Von Karman with super graphic,PC,Hernan,DeSantos,17138 Von Karman,Irvine,Unknown


     pa                                    project_title
0    19                               1 Ethel Coplen Way
1    24                                    21 California
2    51                                   PA 51 D1 South
3    36                          17850 Von Karman Avenue
4    51                                D1 South - Retail
5     9                                 6320 Irvine Blvd
6    34                                 9560 Research Dr
7    33                                           PA #33
8    36                                      17672 Cowan
9    36                                 17138 Von Karman
10   36                             2101 Business Center
11   51                          IUP for 7880 Marine Way
12   39                                            PA 39
13   39                                            PA 39
14   21                                           PA #21
15   36                                 19100 Von Karman
16   51                        

In [21]:
import requests
url2 = "https://legacy.cityofirvine.org/civica/filebank/blobdload.asp?BlobID=11038"
response = requests.get(url, stream=True)

if response.status_code == 200:
    with open('output.pdf', 'wb') as f:
        f.write(response.content)
else:
    print("Failed to download the file")
